In [53]:
import numpy as np
import matplotlib.pyplot as plt
from uncertainties import unumpy

from DataChef import ingredient_functions as ing_funcs
from DataChef import mix_functions as mix_funcs
from DataChef.recipe import Recipe
from DataChef.ingredient import Ingredient

## Welcome to the DataChef tutorial! 📊👨‍🍳
### Let's cook up some fake data 😋

Let's cook up a noisy sine wave! We'll start by collecting our ingredients:

We'll ask for a sine wave with no phase shift, amplitude 4.5, and period 3pi

In [54]:
ing_sine = Ingredient(ing_funcs.sinusoid, "sinusoid", phase=0, amplitude=4.5, period=3*np.pi)

The noise we'll add will be basic gaussian noise with mean 1 and spread 1

In [55]:
ing_gauss = Ingredient(ing_funcs.gaussian, "gaussian noise", mean=1, stdev=1)

Now we'll combine our ingredients into a recipe...

In [56]:
our_first_recipe = Recipe()
our_first_recipe.add_ingredient(ing_sine, mix_funcs.add)
our_first_recipe.add_ingredient(ing_gauss, mix_funcs.add)

And finally, we'll cook them all together, plot the results, and write them to disk!

In [57]:
x = np.linspace(-25,25,200)
our_first_recipe.plot(x)
plt.show()

ValueError: maximum supported dimension for an ndarray is 32, found 200

Alright! That was cool, but let's start again with some different ingredients and try something more ambitious

Here's a parabola, a cubic function, and some uniform white noise. You can see all their parameters in their declarations

In [59]:
ing_parabola = Ingredient(ing_funcs.parabola, "parabola", a=-2, b=0, c=3)
ing_cubic = Ingredient(ing_funcs.cubic, "cubic", a=2, b=0, c=0, d=-5)
ing_uniform = Ingredient(ing_funcs.uniform, "white noise", shift=2, scale=2)

Now add them to a recipe...

In [60]:
rec1 = Recipe()
rec1.add_ingredient(ing_parabola, mix_funcs.add)

We can combine ingredients in more ways than just adding! We'll multiply our white noise on to the parabola then plot it to see our progress

In [61]:
rec1.add_ingredient(ing_uniform, mix_funcs.multiply)
rec1.plot(x, marker='')
plt.show()

TypeError: Recipe.plot() got an unexpected keyword argument 'marker'

Pretty cool, but we can do better! Let's convolve in the cubic function 🤯

In [ ]:
rec1.add_ingredient(ing_cubic, mix_funcs.convolve)
rec1.plot(x, marker='')
plt.show()

I don't know what phenomena this can model but it looks pretty cool!

Let's try to actually model something scientific...

We've built in support for creating analytical models of gravitational wave signals

This will be simulated LISA signal from a BH-BH binary inspiral where each has M=10 Msun and a=0

In [ ]:
gw = Ingredient(ing_funcs.gw_signal, "BH-BH binary")

Let's add it to a recipe and cook up some black holes!

In [ ]:
t = np.linspace(0,1024,1024)
rec2 = Recipe()
rec2.add_ingredient(gw, mix_funcs.add)
rec2.plot(t, marker='')

All real measurements are subject to noise, so we can't forget to consider that

This is characteristic of a random 5% measurement error 

In [ ]:
ing_uniform2 = Ingredient(ing_funcs.uniform, "white noise", shift=0, scale=0.05)
rec2.add_ingredient(ing_uniform2, mix_funcs.multiply)
rec2.plot(t, marker='')

DataChef can also handle error propagation for you! It does this with the help of the __[uncertainties](https://pythonhosted.org/uncertainties/index.html)__ python package. 

You can use the `error_func` keyword when defining an ingredient to provide a function that will generate uncertainties for each data point and propagate those uncertainties as you cook up a recipe. Alternatively, you can set the uncertainties by hand by passing an array to the `yerr` keyword in `Ingredient.eval`.

Here we repeat a recipe from earlier in this tutorial, but now with uncertainties. First, define our ingredients, which will have uncertainties pulled form a uniform distribution. 

In [ ]:
# define some ingredients
ing_parabola = Ingredient(ing_funcs.parabola, "parabola", a=-2, b=0, c=3, 
                            error_func=ing_funcs.uniform, error_func_kwargs={'scale':5})
ing_cubic = Ingredient(ing_funcs.cubic, "cubic", a=2, b=0, c=0, d=-5,
                            error_func=ing_funcs.uniform, error_func_kwargs={'scale':5})
ing_uniform = Ingredient(ing_funcs.uniform, "white noise", shift=2, scale=2,
                            error_func=ing_funcs.uniform, error_func_kwargs={'scale':5})


In [ ]:
# make a simple recipe
rec3 = Recipe()
rec3.add_ingredient(ing_parabola, mix_funcs.add)


# rec3.add_ingredient(ing_uniform, mix_funcs.multiply)
# rec3.add_ingredient(ing_cubic, mix_funcs.convolve)


In [ ]:
# cook the recipe and inspect the errors!
x = np.linspace(1,11,10)
ings_evaluated, cumulative, errs_evaluated, errs_cumulative = rec3.cook_recipe(x)
print(y)